In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("spark-nlp") \
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.11:2.4.5") \
    .getOrCreate()
sc = spark.sparkContext

In [54]:
from pyspark.sql.functions import col, countDistinct, desc, year, month, dayofmonth, hour, dayofweek, unix_timestamp, to_timestamp
import datetime
from pyspark.sql.types import DateType

In [3]:
#load the dataset and shoe the first few columns to confirm the format
congressDF = spark.read.csv("s3://502finalprojbucky/YearlyCongress/*.csv")

In [10]:
congressDF.show(5)

+-------------------+---------------+-------------------+--------------------+--------------------+--------------------+------------------+--------+
|                _c0|            _c1|                _c2|                 _c3|                 _c4|                 _c5|               _c6|dt_stamp|
+-------------------+---------------+-------------------+--------------------+--------------------+--------------------+------------------+--------+
|1207381409809207296|  EliseStefanik|          109579534|2019-12-18T14:25:...|https://www.twitt...|I am excited to d...|Twitter for iPhone|    null|
|1207381362031894537| HouseJudiciary|          246357149|2019-12-18T14:25:...|https://www.twitt...|.@realDonaldTrump...|   Twitter Web App|    null|
|1207381351105732610|CaptClayHiggins|1011053278304591872|2019-12-18T14:25:...|https://www.twitt...|We have a choice,...|   Twitter Web App|    null|
|1207381243408637953|      RepWilson|          234014087|2019-12-18T14:24:...|https://www.twitt...|Today t

In [16]:
congressDF.take(1)

[Row(_c0='1207381409809207296', _c1='EliseStefanik', _c2='109579534', _c3='2019-12-18T14:25:34-05:00', _c4='https://www.twitter.com/EliseStefanik/statuses/1207381409809207296', _c5='I am excited to deliver this significant #Result for Fort Drum and #NY21 after a year of hard work and advocacy @7NewsWatertown #ResultsvsResistance https://www.wwnytv.com/2019/12/18/drum-get-millions-hangar-railhead/', _c6='Twitter for iPhone', dt_stamp=None)]

In [17]:
#convert the datetime stamp to a date column for future analysis
congressDF = congressDF.withColumn('dt_stamp', to_timestamp(col('_c3'), "yyyy-MM-dd'T'HH:mm:ssXXX").cast("timestamp"))
congressDF.show(5)

+-------------------+---------------+-------------------+--------------------+--------------------+--------------------+------------------+-------------------+
|                _c0|            _c1|                _c2|                 _c3|                 _c4|                 _c5|               _c6|           dt_stamp|
+-------------------+---------------+-------------------+--------------------+--------------------+--------------------+------------------+-------------------+
|1207381409809207296|  EliseStefanik|          109579534|2019-12-18T14:25:...|https://www.twitt...|I am excited to d...|Twitter for iPhone|2019-12-18 19:25:34|
|1207381362031894537| HouseJudiciary|          246357149|2019-12-18T14:25:...|https://www.twitt...|.@realDonaldTrump...|   Twitter Web App|2019-12-18 19:25:23|
|1207381351105732610|CaptClayHiggins|1011053278304591872|2019-12-18T14:25:...|https://www.twitt...|We have a choice,...|   Twitter Web App|2019-12-18 19:25:20|
|1207381243408637953|      RepWilson|   

In [63]:
congressDF = congressDF.withColumn("date", congressDF['_c3'].cast(DateType()))
congressDF.show(5)

Py4JJavaError: An error occurred while calling o516.showString.
: java.lang.IllegalStateException: Cannot call methods on a stopped SparkContext.
This stopped SparkContext was created at:

org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
sun.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
sun.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
sun.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.lang.reflect.Constructor.newInstance(Constructor.java:423)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
py4j.Gateway.invoke(Gateway.java:238)
py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
py4j.commands.ConstructorCommand.execute(ConstructorCommand.java:69)
py4j.GatewayConnection.run(GatewayConnection.java:238)
java.lang.Thread.run(Thread.java:748)

In [20]:
#drop columns with null/na values for summary statistics 
congressDF = congressDF.filter(congressDF.date.isNotNull())

In [23]:
dateCounts = congressDF.groupBy("date").count().orderBy(desc('count'))
dateCounts.show(10)

+----------+-----+
|      date|count|
+----------+-----+
|2018-01-30| 7389|
|2019-02-05| 6603|
|2019-12-18| 5827|
|2019-11-20| 5022|
|2019-11-13| 4892|
|2019-12-04| 4865|
|2019-12-12| 4678|
|2019-06-12| 4642|
|2019-07-24| 4583|
|2019-12-11| 4355|
+----------+-----+
only showing top 10 rows



In [27]:
dateCounts.write.csv('congressDateCounts.csv')

In [52]:
hourCounts = congressDF.select(hour("dt_stamp").alias('hour'))\
                     .groupBy("hour").count().orderBy(desc('count'))
hourCounts.show(5)

+----+------+
|hour| count|
+----+------+
|  18|156137|
|  19|156126|
|  20|152310|
|  16|150717|
|  17|150151|
+----+------+
only showing top 5 rows



In [53]:
hourCounts.write.csv('congressHourCounts.csv')

In [57]:
dowCounts = congressDF.select(dayofweek("dt_stamp").alias('dow'))\
                     .groupBy("dow").count().orderBy(desc('count'))
dowCounts.show()  #Sunday is 1

+---+------+
|dow| count|
+---+------+
|  4|363060|
|  5|335706|
|  3|318524|
|  6|272209|
|  2|228901|
|  7|131525|
|  1|103267|
+---+------+



In [58]:
dowCounts.write.csv('congressDOWCounts.csv')

In [59]:
spark.stop()

In [60]:
sc.stop()